# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [87]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [88]:
weatherpy_df = pd.read_csv('../output_data/cities.csv')
weatherpy_df.head(20)


,City,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,port elizabeth,-33.9180,25.5701,66.20,94,90,6.91,ZA,1612731246
1,matara,5.9485,80.5353,74.19,90,0,4.27,LK,1612731280
2,rikitea,-23.1203,-134.9692,77.40,73,97,13.04,PF,1612731506
3,hobart,-42.8794,147.3294,55.40,58,40,13.80,AU,1612730869
4,atar,20.5169,-13.0499,61.97,32,0,9.51,MR,1612731506
5,xinzhou,38.4092,112.7333,26.65,82,0,1.01,CN,1612731506
6,mahendranagar,28.9167,80.3333,55.04,53,0,4.21,NP,1612731506
7,riyadh,24.6877,46.7219,59.00,58,0,11.50,SA,1612730837
8,kefamenanu,-9.4467,124.4781,70.16,88,100,2.01,ID,1612731507
9,kavieng,-2.5744,150.7967,83.37,72,75,7.78,PG,1612731507


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [89]:
locations = weatherpy_df[['Latitude', 'Longitude']]

fig = gmaps.figure()



heat_layer = gmaps.heatmap_layer(locations, weights = weatherpy_df['Humidity (%)'],
                                 dissipating = False, max_intensity=10, point_radius=1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [90]:
#ideal_temp = 
ideal_weather_df = weatherpy_df[(weatherpy_df['Max Temp (F)'] <= 60) &
                                (weatherpy_df['Max Temp (F)'] <= 70) & 
                                (weatherpy_df['Wind Speed (mph)'] <= 10) & 
                                (weatherpy_df['Cloudiness (%)'] <= 25)]
ideal_weather_df.head()

,City,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
5,xinzhou,38.4092,112.7333,26.65,82,0,1.01,CN,1612731506
6,mahendranagar,28.9167,80.3333,55.04,53,0,4.21,NP,1612731506
14,kodiak,57.7900,-152.4072,26.60,74,1,4.61,US,1612731507
24,castrovillari,39.8203,16.2123,55.00,83,14,1.01,IT,1612731508
55,yellowknife,62.4560,-114.3525,-32.01,61,1,4.61,CA,1612730965


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [99]:
ideal_weather_df['Hotel Name'] = ''

for index, row in ideal_weather_df.iterrows():

    lat = row['Latitude']
    lng = row['Longitude']
    #lat_lngg = '38.4092, 112.7333'
    parameters = {
                 'location':f"{lat}, {lng}",
                 'radius': 5000,
                 'type':'hotel',
                 'key': g_key}
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    response = requests.get(base_url, params=parameters)
    hotel_data = response.json()
    #print(hotel_data['results'][0]['name'])
    #print(response.url)
    

    try:
        ideal_weather_df.loc[index, 'Hotel Name'] = hotel_data['results'][0]['name']
        
    except:
        print('No hotel found within 5000 meters radius')
    
ideal_weather_df
#print(json.dumps(hotel_data, indent = 4, sort_keys = True))

C:\Users\gucas\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\gucas\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
5,xinzhou,38.4092,112.7333,26.65,82,0,1.01,CN,1612731506,Xinzhou
6,mahendranagar,28.9167,80.3333,55.04,53,0,4.21,NP,1612731506,Jhalaripipaladi
14,kodiak,57.7900,-152.4072,26.60,74,1,4.61,US,1612731507,Kodiak
24,castrovillari,39.8203,16.2123,55.00,83,14,1.01,IT,1612731508,Castrovillari
55,yellowknife,62.4560,-114.3525,-32.01,61,1,4.61,CA,1612730965,Yellowknife
61,fairbanks,64.8378,-147.7164,-9.00,75,20,4.61,US,1612731282,Fairbanks
108,kutum,14.2000,24.6667,54.45,36,0,7.49,SD,1612731142,Kutum
130,genhe,50.7833,121.5167,-21.10,85,0,2.46,CN,1612731517,Hulunbuir
138,college,64.8569,-147.8028,-9.00,75,20,4.61,US,1612731150,Fairbanks
149,batagay-alyta,67.8006,130.4114,-33.05,78,0,2.44,RU,1612731519,Batagay-Alyta


In [100]:
# hotel_data = response.json()
# #hotel_data
# print(json.dumps(hotel_data, indent = 4, sort_keys = True))

In [45]:
#hotel_data['results'][0]['name']

IndexError: list index out of range

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
